In [ ]:
import pandas as pd

In [ ]:
base_dir = 'car_sales_data'

In [ ]:
import glob

In [ ]:
shape = 0

for file in glob.glob(f'{base_dir}/*.csv'):
    car_sales = pd.read_csv(file)
    shape += car_sales.shape[0]

shape

In [ ]:
# Create a data frame to store
# sales count by state

sales_count_by_state = pd.DataFrame(columns=['state', 'sale_count'])

for file in glob.glob(f'{base_dir}/*.csv'):
    car_sales = pd.read_csv(file)
    sale_count_by_state = car_sales.groupby('state').size().reset_index(name='sale_count')
    sales_count_by_state = pd.concat([sales_count_by_state, sale_count_by_state])


In [ ]:
import polars as pl

In [ ]:
# Read all the files
car_sales = pl.read_csv(f'{base_dir}/*.csv')

In [ ]:
car_sales.shape

In [ ]:
# Get sales by state using polars
sales_by_state = car_sales.group_by('state').agg(
    pl.sum('sale_price').alias('total_sales'),
    pl.count('sale_price').alias('sales_count')
)

In [ ]:
sales_by_state.shape

In [ ]:
pl.Config.set_fmt_float("full")

In [ ]:
sales_by_state.head()

In [ ]:
sales_by_state.sort('total_sales', descending=True)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
session = SparkSession. \
    builder. \
    appName("Car Sales"). \
    getOrCreate()

In [ ]:
car_sales = session. \
    read. \
    csv(base_dir, 
        header=True, 
        inferSchema=True
    )

In [ ]:
car_sales.count()

In [ ]:
from pyspark.sql.functions import count, col

In [ ]:
sales_by_state = car_sales.groupBy('state').agg(
    count('sale_price').alias('sales_count')
)

In [ ]:
sales_by_state.orderBy(col('sales_count').desc())

In [ ]:
sales_by_state.show()